In [1]:
# data manipulation libraries
import numpy as np
import pandas as pd
from ast import literal_eval
from sklearn.model_selection import train_test_split

# pyspark
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.sql import types, functions as F

# random seed for reproducibility
RANDOM_SEED = 15

import shap
import xgboost as xgb
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from cycler import cycler
from sklearn.model_selection import RepeatedKFold
plt.style.use('seaborn-white')

app_name = 'poi-reseller'
conf = (SparkConf()
        .setMaster("yarn-client")
        .setAppName(app_name)
        .set("spark.executor.memory", "8g")
        .set("spark.executor.instances", "1")
        .set("spark.executor.cores", "4")
        .set("spark.driver.memory", "8g")
        .set("spark.yarn.queue", "root.hue_dmp")
        .set("spark.default.parallelism", "8")
        .set("spark.sql.shuffle.partitions", "1000")
        .set("spark.shuffle.service.enabled", "true")
        .set("spark.dynamicAllocation.enabled", "true")
        .set("spark.yarn.driver.memoryOverhead", "4096")
        .set("spark.yarn.executor.memoryOverhead", "4096")
        .set("spark.kryoserializer.buffer.max", "1g")
        .set("spark.dynamicAllocation.minExecutors", "1")
        .set("spark.dynamicAllocation.maxExecutors", "20")
        .set("spark.driver.maxResultSize", "4g")
        .set("spark.sql.crossJoin.enabled", "true")
        .set("spark.hadoop.fs.permissions.umask-mode", "002")
        .set("spark.sql.autoBroadcastJoinThreshold", "-1")
        .set("spark.dynamicAllocation.initialExecutors", "1"))
sc = SparkContext.getOrCreate(conf=conf)
spark = SparkSession(sc)
spark


Bad key backend.qt4 in file /etc/matplotlib/matplotlibrc, line 43 ('backend.qt4 : PyQt4        # PyQt4 | PySide')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.2/matplotlibrc.template
or from the matplotlib source distribution


# pd.options.display.max_columns = None
pd.options.display.max_rows = None

df = spark.sql("""select 
                        b.kabupaten,
                        sum(b.total_trx) as total_trx,
                        sum(b.fea_outlet_under_100m_to_leisure_and_outdoor) fea_outlet_under_100m_to_leisure_and_outdoor,
                        sum(b.fea_outlet_under_100m_to_areas_and_buildings) fea_outlet_under_100m_to_areas_and_buildings,
                        sum(b.fea_outlet_under_100m_to_shopping) fea_outlet_under_100m_to_shopping,
                        sum(b.fea_outlet_under_100m_to_going_out_entertainment) fea_outlet_under_100m_to_going_out_entertainment,
                        sum(b.fea_outlet_under_100m_to_sights_and_museums) fea_outlet_under_100m_to_sights_and_museums,
                        sum(b.fea_outlet_under_100m_to_facilities) fea_outlet_under_100m_to_facilities,
                        sum(b.fea_outlet_under_100m_to_accommodation) fea_outlet_under_100m_to_accommodation,
                        sum(b.fea_outlet_under_100m_to_natural_and_geographical) fea_outlet_under_100m_to_natural_and_geographical,
                        sum(b.fea_outlet_under_100m_to_business_and_services) fea_outlet_under_100m_to_business_and_services,
                        sum(b.fea_outlet_under_100m_to_transport) fea_outlet_under_100m_to_transport,            
                        sum(b.fea_outlet_under_100m_to_eat_and_drink) fea_outlet_under_100m_to_eat_and_drink
                    from
                        mck.ra_digipos_poi_202102 b 
                    where 
                        b.outlet_location_type == 'FISIK' and
                     b.business_partner == 'N'
                     group by 1
""").toPandas()

In [ ]:
print (df[['fea_outlet_under_100m_to_leisure_and_outdoor', 
           'fea_outlet_under_100m_to_areas_and_buildings',
           'fea_outlet_under_100m_to_shopping',
           'fea_outlet_under_100m_to_going_out_entertainment',
           'fea_outlet_under_100m_to_sights_and_museums',
           'fea_outlet_under_100m_to_facilities',
           'fea_outlet_under_100m_to_accommodation',
           'fea_outlet_under_100m_to_natural_and_geographical',
           'fea_outlet_under_100m_to_business_and_services',
           'fea_outlet_under_100m_to_transport',
           'fea_outlet_under_100m_to_eat_and_drink'
          ]].corrwith(df['total_trx']))

In [ ]:
X           = df.drop(['total_trx','kabupaten'], axis=1)
y           = df[['total_trx']]
y           = np.log(y['total_trx']+1)
model       = xgb.XGBRegressor().fit(X, y)
explainer   = shap.Explainer(model)
shap_values = explainer(X)
shap.plots.beeswarm(shap_values, max_display=20)

In [ ]:
from numpy import absolute
model       = xgb.XGBRegressor().fit(X, y)
cv          = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
scores      = cross_val_score(model, X, y, scoring='r2', cv=cv, n_jobs=-1)
scores      = absolute(scores)
print('Mean r2: %.1f (%.1f)' % (scores.mean(), scores.std()) )

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
model = RandomForestRegressor()
model.fit(X, y)

In [ ]:
sklearn.metrics.SCORERS.keys()

In [ ]:
y.median()

In [ ]:
from sklearn import linear_model
model = linear_model.LinearRegression()
model.fit(X, y)

_________________________________________________

In [ ]:
model = RandomForestRegressor()
model.fit(X, y)

In [ ]:
model = DecisionTreeRegressor()
model.fit(X, y)